In [2]:
import pandas as pd

file_path = 'data/xlsx_data/movies_excel.xlsx'
excel_df = pd.read_excel(file_path, sheet_name='Sheet1')  # Specify the sheet name or index

print(len(excel_df))
excel_df.head(10)

3725


,Title,Release Date,Color/B&W,Genre,Language,Country,Rating,Lead Actor,Director Name,Lead Actor FB Likes,Cast FB Likes,Director FB Likes,Movie FB Likes,IMDb Score (1-10),Total Reviews,Duration (min),Gross Revenue,Budget
0,Over the Hill to the Poorhouse,1920-09-15,Black and White,Crime,English,USA,Not Rated,Stephen Carr,Harry F. Millarde,2.0,4,0,0,4.8,1.0,110.0,3000000,100000
1,Metropolis,1927-01-26,Black and White,Drama,German,Germany,Not Rated,Brigitte Helm,Fritz Lang,136.0,203,756,12000,8.3,260.0,145.0,26435,6000000
2,The Broadway Melody,1929-11-11,Black and White,Musical,English,USA,Passed,Anita Page,Harry Beaumont,77.0,109,4,167,6.3,36.0,100.0,2808000,379000
3,42nd Street,1933-08-29,Black and White,Comedy,English,USA,Unrated,Ginger Rogers,Lloyd Bacon,610.0,995,24,439,7.7,65.0,89.0,2300000,439000
4,Top Hat,1935-04-15,Black and White,Comedy,English,USA,Approved,Ginger Rogers,Mark Sandrich,610.0,824,10,1000,7.8,66.0,81.0,3000000,609000
5,Modern Times,1936-10-07,Black and White,Comedy,English,USA,G,Paulette Goddard,Charles Chaplin,309.0,352,0,0,8.6,120.0,87.0,163245,1500000
6,Snow White and the Seven Dwarfs,1937-02-02,Color,Animation,English,USA,Approved,Adriana Caselotti,William Cottrell,82.0,229,0,0,7.7,145.0,83.0,184925485,2000000
7,Gone with the Wind,1939-11-12,Color,Drama,English,USA,G,Hattie McDaniel,Victor Fleming,503.0,1862,149,16000,8.2,157.0,226.0,198655278,3977000
8,The Wizard of Oz,1939-04-09,Black and White,Adventure,English,USA,Passed,Margaret Hamilton,Victor Fleming,695.0,2509,149,14000,8.1,213.0,102.0,22202612,2800000
9,Fantasia,1940-05-18,Color,Animation,English,USA,G,Leopold Stokowski,James Algar,16.0,16,11,3000,7.8,99.0,120.0,76400000,2280000


In [3]:
import json

with open('data/json_data/movies-2010s.json', 'r') as file:
    data = json.load(file)
json_df = pd.json_normalize(data)

print(len(json_df))
json_df.head()

2512


,title,year,cast,genres,href,extract,thumbnail,thumbnail_width,thumbnail_height
0,Winter Day Dreams ft. Franny's Feet and Olivia,2010,"[Franny's Feet, ), Phoebe McAuley, George Buza...","[Animated, Family]",Franny%27s_Feet,Franny's Feet is a Canadian animated children'...,https://upload.wikimedia.org/wikipedia/commons...,200.0,108.0
1,Garbage Dreams,2010,[],[Documentary],Garbage_Dreams,Garbage Dreams is a 2009 feature length docume...,NaN,NaN,NaN
2,Sweetgrass,2010,[],[Documentary],Sweetgrass_(film),Sweetgrass is a 2009 documentary film that fol...,https://upload.wikimedia.org/wikipedia/en/6/62...,260.0,384.0
3,Daybreakers,2010,"[Ethan Hawke, Willem Dafoe, Sam Neill, Claudia...","[Action, Horror, Science Fiction]",Daybreakers,Daybreakers is a 2009 science-fiction action h...,https://upload.wikimedia.org/wikipedia/en/b/b4...,259.0,383.0
4,Leap Year,2010,"[Amy Adams, Matthew Goode, Adam Scott, John Li...","[Comedy, Romance]",Leap_Year_(2010_film),Leap Year is a 2010 romantic comedy directed b...,https://upload.wikimedia.org/wikipedia/en/d/da...,259.0,383.0


In [12]:
csv_df = pd.read_csv("data/csv_data/imdb_top_1000.csv")
print(len(csv_df))
csv_df.head()

1000


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [ ]:
combined_df = csv_df